In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import re, json, glob, argparse
from gensim.corpora import WikiCorpus, Dictionary
from gensim.utils import to_unicode

import argparse 
import json # 
import glob
import logging
import os
import random
import warnings
warnings.filterwarnings(action='ignore')

import timeit
from tqdm import tqdm
import pickle

import numpy as np
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler # Dataset을 보다 간편하게 사용하기 위한 함수
from torch.utils.data.distributed import DistributedSampler 
from fastprogress.fastprogress import master_bar, progress_bar # 학습 과정을 progress_bar로 예쁘게 표현하기 위해 사용하는 tool 
from torch.optim import Adam, AdamW
from torch.utils.data import TensorDataset, RandomSampler, DataLoader
#from transformers import BertTokenizer, BertTokenizerFast, BertForQuestionAnswering
#from transformers import get_linear_schedule_with_warmup

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import  f1_score

import gc

In [ ]:
%cd /content/drive/MyDrive/BIZ&AI_PAPER_11기/세희/memento/task

/content/drive/.shortcut-targets-by-id/1cPIIU0uxZfRWO_8VJ5lEbuVRPz7AvhgA/BIZ&AI_PAPER_11기/세희/memento/task


In [ ]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
def data_load(path):
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = [] 
    start_ids = []
    end_ids = []

    for datas in squad_dict['data']:
        for paragraphs in datas['paragraphs']:
            context = paragraphs['context']
            for document_id
            for qas in paragraphs['qas']:
                question = qas['question']
                for answer in qas['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer['text'])

                    start_index = answer['answer_start']
                    start_ids.append(start_index)

                    answer['text'] = answer['text'].rstrip()
                    
                    end_index = start_index + len(answer['text'])
                    end_ids.append(end_index)                  

    return pd.DataFrame({'contexts' : contexts, 'questions' : questions, 'answers' : answers, 'start_ids': start_ids,'end_ids': end_ids})

In [ ]:
df=data_load('answers_3.json')
df

,contexts,questions,answers,start_ids,end_ids
0,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,2022년 6월 26일 01시 43분에 정통집에서 누구와 함께 행동했나요 ?,석철이,33,36
1,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,2022년 6월 26일 01시 43분에 정통집에서 한 말은 무엇인가요?,석철이랑 제육볶음 먹었다 맛있었다,33,51
2,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,석철이랑 제육볶음 먹었다 맛있었다를 어떤 날짜에 말했나요 ?,2022년 06월 26일,0,13
3,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,석철이랑 제육볶음 먹었다 맛있었다를 말한 시간은 몇 시 인가요 ?,01시 43분,14,21
4,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,석철이랑 제육볶음 먹었다 맛있었다를 말한 장소는 어디인가요 ?,정통집,23,26
...,...,...,...,...,...
646,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,제육볶음 먹었다 고기가 딱딱했다를 말한 시간은 몇 시 인가요?,05시 17분,14,21
647,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,제육볶음 먹었다 고기가 딱딱했다를 말한 장소는 어디인가요?,엘루이피자,23,28
648,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,2020년 12월 14일 05시 17분에 엘루이피자에서 무엇을 했나요?,제육볶음 먹었다,35,43
649,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,2020년 12월 14일 05시 17분에 엘루이피자에서 어떤 감정을 느꼈나요?,고기가 딱딱했다,44,52


## question 내의 "\n" 제거

In [ ]:
def remove_n(text):
    new_string = re.sub(r'\n', '', text)
    return new_string

df['questions']=df['questions'].apply(lambda x: remove_n(x))

In [ ]:
df

,contexts,questions,answers,start_ids,end_ids
0,2021년 11월 18일 05시 25분에 설빙에서 말했다 가게 앞에서 귀여운 강아지...,가게 앞에서 귀여운 강아지를 봤다를 어떤 날짜에 말했나요 ?,2021년 11월 18일,0,13
1,2021년 11월 18일 05시 25분에 설빙에서 말했다 가게 앞에서 귀여운 강아지...,가게 앞에서 귀여운 강아지를 봤다를 말한 시간은 몇 시 인가요 ?,05시 25분,14,21
2,2021년 11월 18일 05시 25분에 설빙에서 말했다 가게 앞에서 귀여운 강아지...,가게 앞에서 귀여운 강아지를 봤다를 말한 장소는 어디인가요 ?,설빙,23,25
3,2021년 11월 18일 05시 25분에 설빙에서 말했다 가게 앞에서 귀여운 강아지...,2021년 11월 18일 05시 25분에 설빙에서 무엇을 했나요 ?,가게 앞에서 귀여운 강아지를 봤다,32,50
4,2021년 11월 18일 05시 25분에 설빙에서 말했다 가게 앞에서 귀여운 강아지...,2021년 11월 18일 05시 25분에 설빙에서 어떤 감정을 느꼈나요?,귀여,39,41
...,...,...,...,...,...
646,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,제육볶음 먹었다 고기가 딱딱했다를 말한 시간은 몇 시 인가요?,05시 17분,14,21
647,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,제육볶음 먹었다 고기가 딱딱했다를 말한 장소는 어디인가요?,엘루이피자,23,28
648,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,2020년 12월 14일 05시 17분에 엘루이피자에서 무엇을 했나요?,제육볶음 먹었다,35,43
649,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,2020년 12월 14일 05시 17분에 엘루이피자에서 어떤 감정을 느꼈나요?,고기가 딱딱했다,44,52


## 결측치 확인

In [ ]:
df.isnull().sum()

contexts     0
questions    0
answers      0
start_ids    0
end_ids      0
dtype: int64

In [ ]:
#dict type 보존되지 않고 str type으로 나오기 때문에 pickle로 저장
df.to_pickle("question&answering.pkl")

In [ ]:
df.to_csv("quesion&answering.csv", index = False)

In [ ]:
final_csv = pd.read_pickle("question&answering.pkl")

In [ ]:
final_csv

,contexts,questions,answers,start_ids,end_ids
0,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,2022년 6월 26일 01시 43분에 정통집에서 누구와 함께 행동했나요 ?,석철이,33,36
1,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,2022년 6월 26일 01시 43분에 정통집에서 한 말은 무엇인가요?,석철이랑 제육볶음 먹었다 맛있었다,33,51
2,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,석철이랑 제육볶음 먹었다 맛있었다를 어떤 날짜에 말했나요 ?,2022년 06월 26일,0,13
3,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,석철이랑 제육볶음 먹었다 맛있었다를 말한 시간은 몇 시 인가요 ?,01시 43분,14,21
4,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...,석철이랑 제육볶음 먹었다 맛있었다를 말한 장소는 어디인가요 ?,정통집,23,26
...,...,...,...,...,...
646,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,제육볶음 먹었다 고기가 딱딱했다를 말한 시간은 몇 시 인가요?,05시 17분,14,21
647,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,제육볶음 먹었다 고기가 딱딱했다를 말한 장소는 어디인가요?,엘루이피자,23,28
648,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,2020년 12월 14일 05시 17분에 엘루이피자에서 무엇을 했나요?,제육볶음 먹었다,35,43
649,2020년 12월 14일 05시 17분에 엘루이피자에서 말했다 제육볶음 먹었다 고기...,2020년 12월 14일 05시 17분에 엘루이피자에서 어떤 감정을 느꼈나요?,고기가 딱딱했다,44,52
